# Implementing VQE for Infinite Square Well potential Potential Quantum System

## Table of Contents
1. [Introduction](#introduction)
2. [How to set up the position space - Grid Discretization](#Grid_Discretization)
3. [Building the Hamiltonian](#Hamiltonian_builder)
4. [Pauli Decomposition](#Pauli_Decomposition)
5. [VQE](#VQE)

<a id="introduction"></a>
## 1. Introduction

This notebook implements VQE (Variational Quantum Eigensolver) for solving the infinite square well potential quantum mechanics problem.


When we first learn quantum mechanics from Griffiths' textbook, we typically use **analytical methods** to find exact solutions to the Schrödinger equation, since it is fundamentally a second-order partial differential equation.

In this notebook, I take a different approach: implementing a **quantum circuit-based method** to solve the same problem using VQE.

### How VQE Works

VQE is a hybrid quantum-classical algorithm that operates in the following way:

1. **Quantum Circuit (Ansatz):** We design a parameterized quantum circuit, called an *ansatz*, that transforms an initial quantum state into a new state determined by circuit parameters.

2. **Quantum Measurement:** The quantum computer measures the expectation value of the Hamiltonian for the current state.

3. **Classical Optimization:** A classical computer uses this measurement result as a cost function and performs gradient descent optimization to adjust the circuit parameters.

4. **Iteration:** We repeat steps until the energy converges to the ground state.

**Key insight:** Up until measurement, the quantum computer does all the computational work. The classical computer then optimizes based on the quantum measurements.

---

📚 **Physics Background**

---
### The Schrödinger Equation

In quantum mechanics, the fundamental equation governing the behavior of particles is the **time-independent Schrödinger equation**:

$$-\frac{\hbar^2}{2m}\frac{d^2\psi}{dx^2} + V(x)\psi(x) = E\psi(x)$$

This can be written more compactly as:

$$H\psi(x) = E\psi(x)$$

where:
- $H = -\frac{\hbar^2}{2m}\frac{d^2}{dx^2} + V(x)$ is the **Hamiltonian operator** (represents total energy: kinetic + potential)
- $\psi(x)$ is the **wavefunction** (describes the quantum state)
- $E$ is the **energy eigenvalue** (observable energy of the system)
- $\hbar$ is the reduced Planck constant
- $m$ is the particle mass
- $V(x)$ is the potential energy

**Our goal:** Find the wavefunction $\psi(x)$ and energy $E$ that satisfy this equation for a given potential $V(x)$. The wavefunction that satisfies this equation represents a **stationary state** of the system, and the lowest energy solution is the **ground state**.

### The Infinite Square Well

Suppose

$$V(x) = \begin{cases}
0 & \text{if } 0 \leq x \leq L \\
\infty & \text{otherwise}
\end{cases}$$

A particle in this potential is completely free, except at the two ends ($x = 0$ and $x = L$), where an infinite force prevents it from escaping. A classical model would be a cart on a frictionless horizontal air track, with perfectly elastic bumpers—it just keeps bouncing back and forth forever.


The infinite square well possesses well-known analytical solutions:

- **Energy eigenvalues:** $E_n = \frac{n^2\pi^2\hbar^2}{2mL^2}$

- **Wavefunctions:** $\psi_n(x) = \sqrt{\frac{2}{L}} \sin\left(\frac{n\pi x}{L}\right)$

**Reference:** Griffiths, D. J. (2005). *Introduction to Quantum Mechanics* (2nd ed.). Pearson Education. pp. 30–35.

---

### Numerical Implementation Note: Discrete Normalization

When simulating this system on a quantum computer (or using linear algebra), we must translate the **continuous wavefunction** $\psi(x)$ into a **discrete state vector** $\vec{v}$.

#### 1. The Conversion: Density vs. Amplitude
* **Continuous World:** Probability is an integral of probability *density*.
    $$\int_0^L |\psi(x)|^2 dx = 1$$
    The unit of $\psi(x)$ is $1/\sqrt{L}$.

* **Discrete World:** Probability is a sum of probability *amplitudes*.
    $$\sum_{i=1}^N |v_i|^2 = 1$$
    The elements $v_i$ must be unitless.

To map between them, we approximate the integral as a Riemann sum $\sum |\psi(x_i)|^2 \Delta x \approx 1$. Therefore, the discrete vector elements must absorb the $\Delta x$ factor:
$$v_i = \psi(x_i) \cdot \sqrt{\Delta x}$$

#### 2. Proof of Exact Normalization
For general functions, this discretization is an approximation. However, for the **Infinite Square Well** sine waves on a symmetric grid, the sum is **exactly 1**.

**The Goal:** Prove that $\sum_{k=1}^N |v_k|^2 = 1$.

Substituting the definitions $dx = \frac{L}{N+1}$ and $\psi(x_k) = \sqrt{\frac{2}{L}}\sin\left(\frac{n\pi k}{N+1}\right)$:

$$\text{Total Prob} = \sum_{k=1}^N \left| \sqrt{\frac{2}{L}} \sin\left(\frac{n \pi k}{N+1}\right) \sqrt{\frac{L}{N+1}} \right|^2$$

The lengths $L$ cancel out, leaving a prefactor dependent only on $N$:

$$\text{Total Prob} = \frac{2}{N+1} \sum_{k=1}^N \sin^2\left(\frac{n \pi k}{N+1}\right)$$

**Step A: Trigonometric Identity**
We use the identity $\sin^2(\theta) = \frac{1 - \cos(2\theta)}{2}$ to split the sum:

$$\sum_{k=1}^N \sin^2(\theta_k) = \underbrace{\sum_{k=1}^N \frac{1}{2}}_{\text{Constant Part}} - \underbrace{\frac{1}{2} \sum_{k=1}^N \cos\left(\frac{2\pi n k}{N+1}\right)}_{\text{Oscillating Part}}$$

1.  **Constant Part:** Summing $1/2$ exactly $N$ times gives $N/2$.
2.  **Oscillating Part:** The term $\sum_{k=1}^N \cos\left(\frac{2\pi n k}{N+1}\right)$ represents the sum of horizontal components of vectors spaced equally around a unit circle (Roots of Unity).     * The sum over a *full* circle ($k=0$ to $N$) is exactly 0 due to symmetry.
    * Our sum is missing the $k=0$ term (where $\cos(0)=1$).
    * Therefore: $1 + \sum_{k=1}^N \cos(\dots) = 0 \implies \sum_{k=1}^N \cos(\dots) = -1$.

**Step B: Final Calculation**
Substitute these values back into the probability equation:

$$\text{Total Prob} = \frac{2}{N+1} \left[ \frac{N}{2} - \frac{1}{2}(-1) \right]$$
$$\text{Total Prob} = \frac{2}{N+1} \left[ \frac{N+1}{2} \right] = 1$$

Thus, the discrete vector is perfectly normalized.


## 2. How to Set Up the Position Space - Grid Discretization

The first step in any simulation is to set up the system. In quantum mechanics, particle position is continuous, but we cannot simulate continuous space computationally—we need to discretize the space into a grid.

For example, consider the infinite square well with width $L$. We choose $N$ grid points, which determines the spatial resolution. Each grid spacing is then $\Delta x = \frac{L}{N+1}$, which we denote as `dx`.

Now, how many qubits do we need to represent $N$ grid points? Since each qubit can represent two states (0 or 1), we need $n_{\text{qubits}} = \lceil \log_2(N) \rceil$ qubits. For example, if $N = 5$ grid points, we need $\lceil \log_2(5) \rceil = 3$ qubits, which can represent up to $2^3 = 8$ states.

### Concrete Example: L = 1 meter, N = 8 grid points

Let's work through a specific example:

**Given:**
- Box length: $L = 1$ m
- Number of grid points: $N = 8$

**Calculate grid spacing:**
$$\Delta x = \frac{L}{N+1} = \frac{1}{8+1} = \frac{1}{9} \approx 0.111 \text{ m}$$

**Grid point positions:**

The grid points are at: $x_i = i \cdot \Delta x$ for $i = 1, 2, \ldots, 8$

**Calculate number of qubits needed:**
$$n_{\text{qubits}} = \lceil \log_2(8) \rceil = \lceil 3 \rceil = 3 \text{ qubits}$$

Since $2^3 = 8$, exactly 3 qubits can represent our 8 grid points:

| Grid Point | Qubit Encoding |
|-----------|----------------|
| $x_1 = 0.111$ | $\|000\rangle$ |
| $x_2 = 0.222$ | $\|001\rangle$ |
| $x_3 = 0.333$ | $\|010\rangle$ |
| $x_4 = 0.444$ | $\|011\rangle$ |
| $x_5 = 0.556$ | $\|100\rangle$ |
| $x_6 = 0.667$ | $\|101\rangle$ |
| $x_7 = 0.778$ | $\|110\rangle$ |
| $x_8 = 0.889$ | $\|111\rangle$ |

**Summary for this example:**
- 8 spatial grid points → 3 qubits
- Each qubit state maps to one position in space
- Hamiltonian is represented as an $8 \times 8$ matrix

### What Happens When N ≠ Power of 2?

When $N$ is **not** a power of 2, we have unused qubit states. For example:

**If N = 5 grid points:**
$$n_{\text{qubits}} = \lceil \log_2(5) \rceil = \lceil 2.32 \rceil = 3 \text{ qubits}$$

This gives us $2^3 = 8$ possible states, but we only use 5 of them:

| Grid Point | State |
|-----------|-------|
| $x_1$ | $\|000\rangle$ |
| $x_2$ | $\|001\rangle$ |
| $x_3$ | $\|010\rangle$ |
| $x_4$ | $\|011\rangle$ |
| $x_5$ | $\|100\rangle$ |
| (unused) | $\|101\rangle$ |
| (unused) | $\|110\rangle$ |
| (unused) | $\|111\rangle$ |

### Hamiltonian Matrix Padding

When $N$ is not a power of 2, the Hamiltonian we build from the spatial grid is naturally an $N \times N$ matrix, but a system of $n_{\text{qubits}}$ qubits lives in a $2^{n_{\text{qubits}}} \times 2^{n_{\text{qubits}}}$ Hilbert space.  
To embed the physical system into the qubit space, we pad the Hamiltonian with a **large energy penalty** on the extra (unphysical) states.

For example, if $N = 5$ grid points and $n_{\text{qubits}} = 3$, then the qubit Hilbert space has dimension $2^3 = 8$.  
The original (physical) Hamiltonian is


\begin{pmatrix}
H_{11} & H_{12} & H_{13} & H_{14} & H_{15} \\
H_{21} & H_{22} & H_{23} & H_{24} & H_{25} \\
H_{31} & H_{32} & H_{33} & H_{34} & H_{35} \\
H_{41} & H_{42} & H_{43} & H_{44} & H_{45} \\
H_{51} & H_{52} & H_{53} & H_{54} & H_{55}
\end{pmatrix},


and we embed it into an $8 \times 8$ matrix as


\begin{pmatrix}
H_{11} & H_{12} & H_{13} & H_{14} & H_{15} & 0      & 0      & 0      \\
H_{21} & H_{22} & H_{23} & H_{24} & H_{25} & 0      & 0      & 0      \\
H_{31} & H_{32} & H_{33} & H_{34} & H_{35} & 0      & 0      & 0      \\
H_{41} & H_{42} & H_{43} & H_{44} & H_{45} & 0      & 0      & 0      \\
H_{51} & H_{52} & H_{53} & H_{54} & H_{55} & 0      & 0      & 0      \\
0      & 0      & 0      & 0      & 0      & \Lambda & 0      & 0      \\
0      & 0      & 0      & 0      & 0      & 0      & \Lambda & 0      \\
0      & 0      & 0      & 0      & 0      & 0      & 0      & \Lambda
\end{pmatrix},


where $\Lambda \gg \text{(typical physical energy scale)}$ is a large energy penalty.

**Interpretation:**

- The upper-left $5 \times 5$ block acts on the **physical** subspace spanned by the valid grid states.
- The remaining basis states (e.g. $\lvert 101\rangle, \lvert 110\rangle, \lvert 111\rangle$) correspond to positions *outside* the box and are given an effectively **infinite potential** via the diagonal entries $\Lambda$.
- Because $\Lambda$ is much larger than any physical eigenvalue, the true low-energy eigenstates and the VQE ground state will naturally avoid these unphysical states and remain in the physical subspace.


## 3. Building the Hamiltonian

Next, we need to build the Hamiltonian $H$, which represents the total energy of the system. Since the Hamiltonian is the sum of kinetic and potential energy, we write:

$$H = T + V$$

where $T$ is the kinetic energy operator and $V$ is the potential energy operator.

For the infinite square well, the potential is $V = 0$ inside the box, so we only need to consider the kinetic energy. Let's see how to construct the kinetic energy matrix using our grid discretization method.

### The Kinetic Energy Operator

In quantum mechanics, the kinetic energy operator is:

$$T = -\frac{\hbar^2}{2m} \frac{d^2}{dx^2}$$

### Discretizing the Second Derivative

When we discretize on a grid, we need to approximate the second derivative $\frac{d^2\psi}{dx^2}$ in terms of function values at grid points. Using a **finite difference approximation**, we have:

$$\frac{d^2\psi}{dx^2} \bigg|_{x} \approx \frac{\psi(x - \Delta x) - 2\psi(x) + \psi(x + \Delta x)}{(\Delta x)^2}$$



### Why This Works: Taylor Series Derivation

To understand why this approximation is valid, let's use Taylor series expansions. For a smooth function $\psi(x)$:

$$\psi(x + \Delta x) = \psi(x) + \psi'(x) \Delta x + \frac{\psi''(x)}{2}(\Delta x)^2 + \frac{\psi'''(x)}{6}(\Delta x)^3 + O((\Delta x)^4)$$

$$\psi(x - \Delta x) = \psi(x) - \psi'(x) \Delta x + \frac{\psi''(x)}{2}(\Delta x)^2 - \frac{\psi'''(x)}{6}(\Delta x)^3 + O((\Delta x)^4)$$

Adding these two expansions:

$$\psi(x + \Delta x) + \psi(x - \Delta x) = 2\psi(x) + \psi''(x)(\Delta x)^2 + O((\Delta x)^4)$$

Rearranging to isolate the second derivative:

$$\psi''(x) = \frac{\psi(x + \Delta x) - 2\psi(x) + \psi(x - \Delta x)}{(\Delta x)^2} + O((\Delta x)^2)$$

**Key observation:** The error term is $O((\Delta x)^2)$, which means the error decreases quadratically as we make the grid finer.

### Building the Kinetic Energy Matrix

On our discretized grid with $N$ points, we can represent the second derivative as a matrix operator. For a vector $\psi = [\psi_1, \psi_2, \ldots, \psi_N]^T$ representing the wavefunction at grid points, the discrete kinetic energy matrix becomes:

$$T = -\frac{\hbar^2}{2m(\Delta x)^2} \begin{pmatrix}
-2 & 1 & 0 & \cdots & 0 \\
1 & -2 & 1 & \cdots & 0 \\
0 & 1 & -2 & \cdots & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
0 & 0 & 0 & \cdots & -2
\end{pmatrix}$$

This **tridiagonal matrix** captures the discrete second derivative: each diagonal element is $-2$, and each off-diagonal element is $1$ (representing neighboring grid points).

Note: Why ψ(0) and ψ(L) don’t appear in the matrix?
the wavefunction must satisfy boundary conditions: \
ψ(0)=0,ψ(L)=0\
The boundary conditions are hard-coded into the Laplacian by omitting ghost grid values.

#### Concrete Example: N = 4 Grid Points

Let's see what this matrix looks like concretely with $N = 4$ grid points: $\psi_1, \psi_2, \psi_3, \psi_4$.

The kinetic energy matrix is:

$$T = -\frac{\hbar^2}{2m(\Delta x)^2} \begin{pmatrix}
-2 & 1 & 0 & 0 \\
1 & -2 & 1 & 0 \\
0 & 1 & -2 & 1 \\
0 & 0 & 1 & -2
\end{pmatrix}$$

**Why does it have this structure?** When we apply $T$ to the wavefunction vector $\psi$, the action on each grid point is:

$$T \psi = -\frac{\hbar^2}{2m(\Delta x)^2} \begin{pmatrix}
-2\psi_1 + \psi_2 \\
\psi_1 - 2\psi_2 + \psi_3 \\
\psi_2 - 2\psi_3 + \psi_4 \\
\psi_3 - 2\psi_4
\end{pmatrix}$$

Notice that each row encodes the finite difference approximation for that grid point:
- **Row 1:** $-2\psi_1 + \psi_2 = \frac{d^2\psi}{dx^2}\big|_1 \cdot (\Delta x)^2$ (point 1 with right neighbor)
- **Row 2:** $\psi_1 - 2\psi_2 + \psi_3 = \frac{d^2\psi}{dx^2}\big|_2 \cdot (\Delta x)^2$ (point 2 with both neighbors)
- **Row 3:** $\psi_2 - 2\psi_3 + \psi_4 = \frac{d^2\psi}{dx^2}\big|_3 \cdot (\Delta x)^2$ (point 3 with both neighbors)
- **Row 4:** $\psi_3 - 2\psi_4 = \frac{d^2\psi}{dx^2}\big|_4 \cdot (\Delta x)^2$ (point 4 with left neighbor)

This is exactly our finite difference formula applied at each point!

**General pattern for N grid points:**
- Diagonal elements: always $-2$
- One element above/below diagonal: always $1$
- All other elements: $0$ (no connection between distant points)

---
The potential in the infinite square well is defined as:

$$V(x) = \begin{cases}
0 & \text{if } 0 \leq x \leq L \\
\infty & \text{otherwise}
\end{cases}$$

Inside the box ($0 \leq x \leq L$), the potential is zero, meaning the particle moves freely without any force. Outside the box, the potential is infinite, creating an impenetrable barrier that confines the particle.

**This simplification is crucial:** Since the potential is zero inside the box, the Schrödinger equation reduces to:

$$-\frac{\hbar^2}{2m}\frac{d^2\psi}{dx^2} = E\psi(x)$$

We no longer need to include the $V(x)$ term, which makes this problem analytically solvable. This is why the infinite square well is such an elegant test case—the mathematics is simple enough to solve by hand, yet rich enough to illustrate fundamental quantum concepts.



## 4. Pauli Decomposition of the Hamiltonian

### 4.1 Theoretical Background

Any $2^n \times 2^n$ Hermitian matrix (including Hamiltonians) can be uniquely decomposed into a sum of tensor products of Pauli matrices:

$$H = \sum_{\alpha} c_{\alpha} P_{\alpha}$$

where:
- $P_{\alpha} \in \{I, X, Y, Z\}^{\otimes n}$ are **Pauli strings** (combinations of Pauli operators on $n$ qubits)
- $c_{\alpha} \in \mathbb{R}$ are **real coefficients**

**Example of Pauli strings for 3 qubits:**
- $IZZ$ (identity on qubit 0, Z on qubits 1 and 2)
- $XYI$ (X on qubit 0, Y on qubit 1, identity on qubit 2)
- $ZZZ$ (Z on all three qubits)

The Pauli matrices form an **orthonormal basis** under the Hilbert-Schmidt inner product:

$$\text{Tr}(P_{\alpha} P_{\beta}) = 2^n \delta_{\alpha\beta}$$

where $\delta_{\alpha\beta}$ is the Kronecker delta (1 if $\alpha = \beta$, 0 otherwise).

---

### 4.2 Derivation of the Pauli Coefficient Formula

#### Why $c_{\alpha} = \frac{1}{2^n} \text{Tr}(P_{\alpha} H)$?

**Starting point:** We know that Pauli operators form an orthonormal basis:

$$H = \sum_{\alpha} c_{\alpha} P_{\alpha}$$

**Goal:** Find $c_{\alpha}$ for a specific Pauli string $P_{\alpha}$.

**Step 1:** Multiply both sides by $P_{\beta}$ and take the trace:

$$\text{Tr}(P_{\beta} H) = \text{Tr}\left(P_{\beta} \sum_{\alpha} c_{\alpha} P_{\alpha}\right)$$

$$\text{Tr}(P_{\beta} H) = \sum_{\alpha} c_{\alpha} \text{Tr}(P_{\beta} P_{\alpha})$$

**Step 2:** Use the orthonormality of Pauli matrices:

The Pauli matrices satisfy the Hilbert-Schmidt inner product:

$$\text{Tr}(P_{\alpha} P_{\beta}) = 2^n \delta_{\alpha\beta}$$

where $\delta_{\alpha\beta} = 1$ if $\alpha = \beta$, and $0$ otherwise.

**Step 3:** Apply this orthonormality property:

$$\text{Tr}(P_{\beta} H) = \sum_{\alpha} c_{\alpha} \cdot 2^n \delta_{\alpha\beta} = 2^n c_{\beta}$$

The sum collapses because $\delta_{\alpha\beta}$ is zero for all terms except when $\alpha = \beta$.

**Step 4:** Solve for $c_{\beta}$:

$$c_{\beta} = \frac{1}{2^n} \text{Tr}(P_{\beta} H)$$

Or equivalently (dropping the subscript):

$$\boxed{c_{\alpha} = \frac{1}{2^n} \text{Tr}(P_{\alpha} H)}$$

**Intuition:** This formula extracts the "component" of $H$ along each Pauli direction, similar to how we compute coefficients in any orthonormal basis expansion (like Fourier analysis).

---

### 4.3 Practical Example: Computing Pauli Coefficients

**Given:** A $2 \times 2$ single-qubit Hamiltonian

$$H = \begin{pmatrix} 2 & 1 \\ 1 & 0 \end{pmatrix}$$

**Find:** The coefficients for $I, X, Y, Z$.

**Recall Pauli matrices:**
- $I = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix}$
- $X = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}$
- $Y = \begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix}$
- $Z = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}$

**Calculate:**

$$c_I = \frac{1}{2^1} \text{Tr}(I \cdot H) = \frac{1}{2} \text{Tr}\begin{pmatrix} 2 & 1 \\ 1 & 0 \end{pmatrix} = \frac{1}{2}(2 + 0) = 1$$

$$c_X = \frac{1}{2} \text{Tr}(X \cdot H) = \frac{1}{2} \text{Tr}\begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}\begin{pmatrix} 2 & 1 \\ 1 & 0 \end{pmatrix} = \frac{1}{2} \text{Tr}\begin{pmatrix} 1 & 0 \\ 2 & 1 \end{pmatrix} = \frac{1}{2}(1 + 1) = 1$$

$$c_Z = \frac{1}{2} \text{Tr}(Z \cdot H) = \frac{1}{2} \text{Tr}\begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}\begin{pmatrix} 2 & 1 \\ 1 & 0 \end{pmatrix} = \frac{1}{2} \text{Tr}\begin{pmatrix} 2 & 1 \\ -1 & 0 \end{pmatrix} = \frac{1}{2}(2 + 0) = 1$$

**Result:**

$$H = I + X + Z$$

**Verification:** $I + X + Z = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix} + \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix} + \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix} = \begin{pmatrix} 2 & 1 \\ 1 & 0 \end{pmatrix} = H$ ✓

---

### 4.4 Decomposition Algorithm

To find the coefficient for each Pauli string:

**Algorithm:**
1. For each possible Pauli string $P_{\alpha}$ (there are $4^n$ total)
2. Compute the trace: $\text{Tr}(P_{\alpha} H)$
3. Divide by $2^n$ to get the coefficient $c_{\alpha}$
4. Keep only coefficients with magnitude above threshold: $|c_{\alpha}| > \epsilon$

---

### 4.5 Why Pauli Decomposition Matters for VQE

**On a quantum computer**, we cannot directly measure the Hamiltonian. Instead, we:

1. **Decompose** $H$ into Pauli strings
2. **Measure** each Pauli term separately
3. **Combine** results to get the total energy

**Example:** For a 2-qubit Hamiltonian:

$$H = 0.5 \cdot ZZ + 0.3 \cdot ZI + 0.2 \cdot IX$$

We measure three separate things:
- Measure $\langle ZZ \rangle$ (Z on both qubits) → multiply by 0.5
- Measure $\langle ZI \rangle$ (Z on qubit 0, nothing on qubit 1) → multiply by 0.3
- Measure $\langle IX \rangle$ (nothing on qubit 0, X on qubit 1) → multiply by 0.2

$$E = 0.5 \langle ZZ \rangle + 0.3 \langle ZI \rangle + 0.2 \langle IX \rangle$$


---


## 4.6 Measuring Quantum Observables: $\langle Z \rangle, \langle X \rangle, \langle Y \rangle$

In quantum computing hardware, measurements are natively performed in the **Computational Basis** (the $Z$-basis). This means we project the quantum state onto the eigenvectors of the Pauli-$Z$ operator: $|0\rangle$ and $|1\rangle$.

However, we often need to measure expectation values for other operators, such as $\langle X \rangle$ or $\langle Y \rangle$. To do this, we must perform a **Change of Basis** operation on the state vector *before* the measurement.

### 4.6.1 The General Theory: Constructing the Measurement Gate

To measure in a specific basis (let's call it the $A$-basis), we need to transform our state vector from its standard representation ($Z$-basis) into the $A$-basis representation.

If the operator $A$ has eigenvectors $|a_0\rangle$ and $|a_1\rangle$ (written in the $Z$-basis), we construct an **Eigenbasis Matrix** $V$ by placing these eigenvectors as columns:

$$V = \begin{pmatrix} |a_0\rangle & |a_1\rangle \end{pmatrix}$$

This matrix $V$ translates a vector **from** the $A$-basis **to** the $Z$-basis. Therefore, to translate our state **into** the $A$-basis for measurement, we apply the inverse:

$$\text{Measurement Gate} = V^{-1} = V^{\dagger}$$

*Note: Since change-of-basis matrices in quantum mechanics are Unitary, the inverse is equal to the Conjugate Transpose ($V^{\dagger}$).*

---

### 4.6.2 Measuring $\langle X \rangle$

The eigenstates of the Pauli-$X$ operator are $|+\rangle$ and $|-\rangle$:
* $|+\rangle = \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$
* $|-\rangle = \frac{1}{\sqrt{2}}(|0\rangle - |1\rangle)$

We construct the matrix $V_X$:
$$V_X = \frac{1}{\sqrt{2}} \begin{pmatrix} 1 & 1 \\ 1 & -1 \end{pmatrix}$$

The inverse is simply the **Hadamard Gate ($H$)**:
$$V_X^{\dagger} = H = \frac{1}{\sqrt{2}} \begin{pmatrix} 1 & 1 \\ 1 & -1 \end{pmatrix}$$

**Protocol:** Apply **$H$** before measuring.

---

### 4.6.3 Measuring $\langle Y \rangle$

The eigenstates of the Pauli-$Y$ operator are $|+i\rangle$ and $|-i\rangle$:
* $|+i\rangle = \frac{1}{\sqrt{2}}(|0\rangle + i|1\rangle)$ (Eigenvalue +1)
* $|-i\rangle = \frac{1}{\sqrt{2}}(|0\rangle - i|1\rangle)$ (Eigenvalue -1)

We construct the matrix $V_Y$:
$$V_Y = \frac{1}{\sqrt{2}} \begin{pmatrix} 1 & 1 \\ i & -i \end{pmatrix}$$

To find the circuit gate, we take the Conjugate Transpose ($V_Y^{\dagger}$):
$$V_Y^{\dagger} = \frac{1}{\sqrt{2}} \begin{pmatrix} 1 & -i \\ 1 & i \end{pmatrix}$$

In terms of standard quantum gates, this matrix is equivalent to **$HS^{\dagger}$**:
1.  **$S^{\dagger}$** rotates the $Y$-axis to the $X$-axis.
2.  **$H$** rotates the $X$-axis to the $Z$-axis.

**Protocol:** Apply **$S^{\dagger}$** followed by **$H$** before measuring.

*Note: If you derive the matrix manually using different phase factors for eigenvectors (e.g., using $-i$), the resulting matrix might look different mathematically. However, because probabilities rely on the absolute square of the amplitude ($|\alpha|^2$), global phase factors disappear, and the physical results remain identical to the standard $HS^{\dagger}$ method.*



## 5. VQE (Variational Quantum Eigensolver)

### 5.1 The Big Picture: Training the Quantum Computer
When we learned quantum mechanics analytically (Section 1), we solved differential equations to find the wavefunction $\psi(x)$.
In this section, we take a radically different approach. We are effectively training the quantum computer to become a **function generator**.

By tuning the parameters ($\theta$) of our quantum circuit, we are physically forcing the probability amplitudes of the qubits to morph into the shape of the sine wave solution.

### 5.2 The Transformation Process
The logic of our VQE implementation is as follows:

**1. Input State**

The quantum computer always initializes qubits to the zero state:

$$
|\psi_{initial}\rangle = |00\dots0\rangle = \begin{pmatrix} 1 \\ 0 \\ \vdots \\ 0 \end{pmatrix}
$$

*Physically, this represents a particle trapped entirely at the first grid point $x_1$.*

**2. The Ansatz (The "Knobs")**

We apply a parameterized circuit $U(\theta)$. This is a sequence of rotations and entanglements that "smears" the probability out across the other states.

**3. Output State**

If the VQE optimization is successful, the final state vector amplitudes will match the values of our discretized wavefunction:

$$
|\psi_{final}\rangle = U(\theta)|00\dots0\rangle \approx \begin{pmatrix} v_1 \\ v_2 \\ \vdots \\ v_N \end{pmatrix}
$$

where $v_k \approx \psi(x_k) \cdot \sqrt{\Delta x}$.

### 5.3 Mapping Qubits to Position
It is crucial to understand exactly what each qubit state represents in our physical grid. For a system with $N=4$ grid points (requires 2 qubits), the mapping is:

| Qubit State | Vector Index | Physical Position | Target Amplitude ($\alpha$) |
| :--- | :--- | :--- | :--- |
| $|00\rangle$ | 0 | $x_1$ | $\sqrt{\Delta x} \cdot \psi(x_1)$ |
| $|01\rangle$ | 1 | $x_2$ | $\sqrt{\Delta x} \cdot \psi(x_2)$ |
| $|10\rangle$ | 2 | $x_3$ | $\sqrt{\Delta x} \cdot \psi(x_3)$ |
| $|11\rangle$ | 3 | $x_4$ | $\sqrt{\Delta x} \cdot \psi(x_4)$ |

### 5.4 Simulation Note: The `StatevectorEstimator`
In this notebook, we use Qiskit's `StatevectorEstimator`. It is important to note that this is an **idealized simulation**.

* **Real Hardware:** A real quantum computer cannot "see" the vector. It must measure shots (0s and 1s) and build a statistical average, which introduces sampling noise.
* **StatevectorEstimator:** This method calculates the exact expectation value $\langle \psi | H | \psi \rangle$ using matrix multiplication (linear algebra). This allows us to debug our physics logic and ansatz design without worrying about statistical noise.

### 5.5 Global Phase Warning
When visualizing the results, you may notice the VQE solution appears "flipped" (e.g., negative amplitudes where you expect positive ones). This is due to **Global Phase**. In quantum mechanics, $|\psi\rangle$ and $-|\psi\rangle$ represent the exact same physical state because their probability distributions ($|\psi|^2$) are identical.

# Implementation

## Install libraries

In [ ]:
# Install required packages for Colab
!pip install qiskit qiskit-aer qiskit-algorithms pylatexenc
# Check what's already installed
import numpy as np
import scipy
import matplotlib
import qiskit

print(f"✓ NumPy: {np.__version__}")
print(f"✓ SciPy: {scipy.__version__}")
print(f"✓ Matplotlib: {matplotlib.__version__}")
print(f"✓ Qiskit: {qiskit.__version__}")

✓ NumPy: 2.0.2
✓ SciPy: 1.16.3
✓ Matplotlib: 3.10.0
✓ Qiskit: 2.2.3


## Grid Discretization

In [ ]:
import numpy as np
from typing import Callable, Tuple, Optional

class GridDiscretization:
  """
  Implements position-space grid discretization for quantum system.

  The continuous Hamiltonian is discretized on a spatial grid using finite difference methods

  Attributes:
    L (float): Length of the box (system domain)
    N (int): Number of grid points (determines the spatial resolution)
    dx (float): Grid spacing = L/(N+1)
    x_grid (np.ndarray): Array of grid point positions
    n_qubits (int): Number of qubits needed to represent the grid = ceil(log2(N))
  """
  def __init__(self, L: float, N: int, hbar: float = 1.0, m: float = 1.0):
    """
    Initialize the GridDiscretization object

    Args:
      L (float): Length of the box (system domain)
      N (int): Number of grid points (determines the spatial resolution)
      hbar: (Optional) Reduced Planck (default: 1.0)
      m: (Optional) Mass (default: 1.0)
    """
    self.L=L
    self.N=N
    self.hbar=hbar
    self.m=m
    self.dx=L/(N+1)
    self.x_grid=np.linspace(self.dx, L-self.dx, N)
    self.n_qubits=int(np.ceil(np.log2(N)))
    self.hilbert_space_dim=2**self.n_qubits

  def build_kinetic_energy(self) -> np.ndarray:
    """
    Build kinetic energy matrix using finite difference approximation.

    Returns:
      Kinetic energy matrix (N x N) with unit of ℏ²/(2m·dx²)
    """

    # Coefficient for kinetic energy operator
    coeff = -self.hbar**2/(2*self.m*self.dx**2)
    kinetic = np.zeros((self.N,self.N))
    for i in range(self.N):
      kinetic[i,i]=-2.0
      if i>0:
        kinetic[i,i-1]=1.0
      if i<self.N-1:
        kinetic[i,i+1]=1.0

    return coeff * kinetic

  def build_potential_energy(self, potential_func: Callable[[np.ndarray], np.ndarray]) -> np.ndarray:
      """
      Build potential energy matrix using given potential function.

      The matrix is diagonal because in quantum mechanics, the potential energy
      at each position is independent—it only depends on that position's location,
      not on neighboring positions. This is why we get a diagonal matrix: each
      diagonal element V(xᵢ) represents the potential at position xᵢ, and all
      off-diagonal elements are zero (no coupling between positions).

      Note: For the infinite square well, the potential is V = 0 everywhere inside
      the box, so this method returns a zero matrix. However, we include this method
      here as a general framework so that in the future we can easily simulate
      different types of potentials (e.g., harmonic oscillator, finite well, etc.)
      without rewriting the code.

      Args:
        potential_func: A function that takes grid positions (np.ndarray) and
                        returns potential values at those positions (np.ndarray)

      Returns:
        Potential energy matrix (N x N diagonal matrix)
      """
      V = potential_func(self.x_grid)
      return np.diag(V)


  def build_hamiltonian(self, potential_func: Optional[Callable]= None) -> np.ndarray:
    """
    Build full Hamiltonian matrix using kinetic and potential energy matrices.

    Args:
      potential_func: (Optional) A function that takes grid positions (np.ndarray)
                      and returns potential values. If None, assumes V=0

    Returns:
      Full Hamiltonian matrix (N x N)
    """
    H= self.build_kinetic_energy()
    if potential_func is not None:
      H+= self.build_potential_energy(potential_func)
    return H

  def analytical_infinite_well(self,n:int) -> Tuple[float,np.ndarray]:
    """
    Get analytical solution for infinite square well.
    Args:
      n: Quantum Number (n=1 is ground state and when n>1, the wavefunction is in excited state)
    Returns:
      Energy eigenvalue and normalized wavefunction at grid points
    """
    #Energy Eigenvalue
    E_n=(n*np.pi*self.hbar)**2/(2 * self.m * self.L**2)
    # Wavefunction (normalized)
    psi_n = np.sqrt(2/self.L)*np.sin(n*np.pi*self.x_grid/self.L)
    # Discrete-normalized version: sum_i |psi[i]|^2 ≈ 1
    psi_n_discrete = np.sqrt(self.dx)*psi_n
    return E_n, psi_n_discrete

  def pad_to_qubit_dimension(self, matrix: np.ndarray, penalty_factor: float = 1e3) -> np.ndarray:
    """
    Pad matrix to size 2^n_qubits × 2^n_qubits for qubit encoding.

    The top-left N×N block contains the physical Hamiltonian.
    The remaining basis states (unphysical) are given a large
    diagonal energy penalty Λ so that low-energy eigenstates
    stay inside the physical subspace.

    Args:
        matrix: Original matrix of size N × N
        penalty_factor: Multiplier that sets how large the
                        energy penalty Λ is relative to the
                        physical energy scale.

    Returns:
        Padded matrix of size 2^n_qubits × 2^n_qubits
    """

    # If it's already the right size, do nothing
    if matrix.shape[0] == self.hilbert_space_dim:
        return matrix

    # Start from an all-zero matrix
    padded = np.zeros((self.hilbert_space_dim, self.hilbert_space_dim), dtype=matrix.dtype)

    # Embed the physical Hamiltonian in the top-left corner
    n = matrix.shape[0]
    padded[:n, :n] = matrix

    # Choose an energy scale based on the Hamiltonian
    max_entry = np.max(np.abs(matrix))
    if max_entry == 0:
        max_entry = 1.0
    Lambda = penalty_factor * max_entry

    # Put Λ on the diagonal of the unused subspace
    padded[n:, n:] = np.eye(self.hilbert_space_dim - n) * Lambda

    return padded


  def get_grid_info(self) -> dict:
    """
    Get information about the discretization grid.

    Returns:
        Dictionary containing grid parameters
    """
    return {
        'L': self.L,
        'N': self.N,
        'dx': self.dx,
        'n_qubits': self.n_qubits,
        'hilbert_dim': self.hilbert_space_dim,
        'x_min': self.x_grid[0],
        'x_max': self.x_grid[-1],
        'hbar': self.hbar,
        'm': self.m
    }

__all__ = ["GridDiscretization"]




## Pauli Decomposition Manual Creation

In [ ]:
import numpy as np
from typing import Callable, Dict, List, Optional, Tuple

class PauliDecomposition:
  """Pauli-basis decomposition of an n-qubit Hermitian matrix.
  This implements the theory from Section 4 :
  H = sum_α c_α P_α, with c_α = Tr(P_α H) / 2^n
  Where P_α ∈ {I, X, Y, Z}^{⊗ n} are Pauli strings.
  This implementation is intentionally "from scratch" and only depends on Numpy, so you can see the mechanics explicitly.
  An optional helper is provided to convert the result into a Qiskit SparsePauliOp if you want to plug it into VQE later.
  """

  # Single_qubit Pauli matrices in the computational Z basis
  _single_qubit_paulis: Dict[str,np.ndarray]= {"I":np.array([[1,0],[0,1]],dtype=complex),
                                               "X":np.array([[0,1],[1,0]],dtype=complex),
                                               "Y":np.array([[0,-1j],[1j,0]],dtype=complex),
                                               "Z":np.array([[1,0],[0,-1]],dtype=complex)
                                               }
  def __init__(self, n_qubits: Optional[int]=None,tol: float =1e-10) ->None:
    """
    Initialize a PauliDecomposition helper.
    Args:

      n_qubits: Optional number of qubits. If None, it will be inferred from the matrix dimension dim=2**n.

      tol: Coefficients with |c_α| < tol are dropped from the output list.
    """
    self.n_qubits: Optional[int] = n_qubits
    self.tol: float= tol
    #Cache for already_built Pauli matrices to avoid recomputing
    self._pauli_cache: Dict[str,np.ndarray]={}

  def _get_n_qubits(self, H: np.ndarray) -> int:
      """Return n_qubits, inferring it from H if necessary."""
      if H.ndim != 2 or H.shape[0] != H.shape[1]:
          raise ValueError("Hamiltonian must be a square matrix.")

      dim = H.shape[0]

      if self.n_qubits is None:
          # infer and store
          n = int(np.round(np.log2(dim)))
          if 2**n != dim:
              raise ValueError(
                  f"Matrix dimension {dim} is not a power of 2; "
                  "cannot interpret as an n-qubit operator."
              )
          self.n_qubits = n
      else:
          # just consistency check
          if dim != 2**self.n_qubits:
              raise ValueError(
                  f"Provided n_qubits {self.n_qubits} does not match "
                  f"matrix dimension {dim}."
              )

      return self.n_qubits


  # Pauli basis construction
  def generate_pauli_strings(self, n_qubits:Optional[int]=None)->List[str]:
    """
    Generate all possible Pauli strings for n_qubits.
    """
    import itertools

    if n_qubits is None:
      if self.n_qubits is None:
        raise ValueError("n_qubits is not set and cannot be inferred.")
      n_qubits= self.n_qubits
    labels= ["I","X","Y","Z"]
    return ["".join(p) for p in itertools.product(labels,repeat=n_qubits)]

  def pauli_matrix(self, label: str) -> np.ndarray:
      """
      Build the matrix representation of an n-qubit Pauli string.

      Each character in `label` must be one of {"I", "X", "Y", "Z"}.
      The length of `label` determines the number of qubits:
      - len(label) = 1  →  1-qubit operator
      - len(label) = 2  →  2-qubit operator
      - etc.

      Args:
          label: Pauli string specifying the operator, e.g.
              - "Z"
              - "XI"
              - "YZX"
              - "IIIZ"

      Returns:
          A NumPy array of shape (2**len(label), 2**len(label)) with dtype=complex
          containing the matrix representation of the Pauli string in the
          computational (Z) basis.

      Example:
          >>> dec = PauliDecomposition(n_qubits=2)
          >>> P = dec.pauli_matrix("ZI")
          >>> P.shape
          (4, 4)
          >>> P
          array([[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
                [ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j],
                [ 0.+0.j,  0.+0.j, -1.+0.j,  0.+0.j],
                [ 0.+0.j,  0.+0.j,  0.+0.j, -1.+0.j]])

      Notes:
          - For performance, matrices are cached in `self._pauli_cache`, so
            calling `pauli_matrix("ZZI")` multiple times reuses the same array.
      """
      if label in self._pauli_cache:
          return self._pauli_cache[label]

      # Initialize with 1×1 identity so the first kron works
      mat = np.array([1], dtype=complex)
      for ch in label:
          if ch not in self._single_qubit_paulis:
              raise ValueError(f"Unknown Pauli symbol '{ch}' in label {label!r}")
          mat = np.kron(mat, self._single_qubit_paulis[ch])

      self._pauli_cache[label] = mat
      return mat


  # ------------------------------------------------------------------
  # Main decomposition routine
  # ------------------------------------------------------------------
  def decompose(self, H: np.ndarray) -> List[Tuple[str, float]]:
      """
      Decompose a Hermitian matrix H into a sum of Pauli strings.

      Uses the formula derived in Section 4.2:
          c_α = Tr(P_α H) / 2**n
      where {P_α} is the full n-qubit Pauli basis:
          P_α ∈ {I, X, Y, Z}^{⊗ n}.

      Args:
          H:
              A (2**n) × (2**n) Hermitian NumPy array representing an n-qubit
              operator. In this notebook, this is typically the padded Hamiltonian
              returned by `GridDiscretization.pad_to_qubit_dimension()`.

      Returns:
          A list of (pauli_label, coefficient) pairs, where:
              - pauli_label (str): Pauli string such as "ZZII", "IX", or "I".
              - coefficient (float): Real coefficient c_α. Terms with
                |c_α| ≤ self.tol are omitted.

          The returned expansion satisfies (up to numerical tolerance):
              H ≈ ∑_α c_α P_α

      Example:
          A simple 1-qubit Hamiltonian:

          >>> H = np.array([[2, 1],
          ...               [1, 0]], dtype=complex)

          Analytically, we know:
              H = I + X + Z

          >>> dec = PauliDecomposition(n_qubits=1)
          >>> terms = dec.decompose(H)
          >>> terms
          [('I', 1.0), ('X', 1.0), ('Z', 1.0)]

          For a 2-qubit operator:

          >>> H2 = np.diag([1, 2, 3, 4]).astype(complex)
          >>> dec2 = PauliDecomposition(n_qubits=2)
          >>> dec2.decompose(H2)[:3]   # first few terms
          [('II', 2.5), ('ZI', -1.0), ('IZ', -0.5)]
          # (Exact values may differ slightly due to floating-point rounding.)

      Raises:
          ValueError:
              - If H is not a square matrix,
              - If its dimension is not a power of 2 (cannot map to n qubits),
              - Or if H is not Hermitian within numerical tolerance.
      """
      H = np.asarray(H, dtype=complex)
      n = self._get_n_qubits(H)

      # Optional: sanity-check Hermiticity
      if not np.allclose(H, H.conj().T, atol=1e-8):
          raise ValueError("H is not Hermitian within numerical tolerance.")

      dim_factor = 2 ** n
      terms: List[Tuple[str, float]] = []

      for label in self.generate_pauli_strings(n):
          P = self.pauli_matrix(label)
          # c_α = Tr(P_α H) / 2^n
          c = np.trace(P @ H) / dim_factor

          # For Hermitian H and Pauli P, c should be real
          c_real = float(np.real_if_close(c, tol=1e-9))
          if abs(c_real) > self.tol:
              terms.append((label, c_real))

      return terms

__all__ = ["PauliDecomposition"]







## Pauli Decomposition using Qiskit

In [ ]:
import numpy as np
from qiskit.quantum_info import Operator, SparsePauliOp

def pauli_dict_from_qiskit(H: np.ndarray, tol: float = 1e-10):
    """
    Dense matrix H -> Qiskit SparsePauliOp -> {label: coeff} dict

    example output:
          {
            "III": 81+0j,
            "IIX": -40.5+0j,
            ...
          }


    """
    H_op = Operator(H)
    sp = SparsePauliOp.from_operator(H_op)

    # Get labels and coeffs
    try:
        labels = list(sp.paulis.to_labels())
    except AttributeError:
        # fallback for older versions
        labels, coeffs = zip(*sp.to_list())
        coeffs = np.array(coeffs, dtype=complex)
        return {
            l: c for l, c in zip(labels, coeffs)
            if abs(c) > tol
        }

    coeffs = np.asarray(sp.coeffs, dtype=complex)

    return {
        l: c for l, c in zip(labels, coeffs)
        if abs(c) > tol
    }

def pauli_dict_from_custom(H: np.ndarray, tol: float = 1e-10):
    """
    Dense matrix H -> your PauliDecomposition -> {label: coeff} dict
    """
    dec = PauliDecomposition(tol=tol)
    terms = dec.decompose(H)  # returns List[(label, float)]

    return {label: float(coeff) for label, coeff in terms}

def compare_pauli_decompositions(H: np.ndarray, tol: float = 1e-8):
    """
    Compare Qiskit vs your PauliDecomposition on the same H.
    Prints any mismatches.
    """
    d_q = pauli_dict_from_qiskit(H, tol=tol)
    d_c = pauli_dict_from_custom(H, tol=tol)

    # Union of labels
    all_labels = sorted(set(d_q.keys()) | set(d_c.keys()))

    mismatches = []
    for label in all_labels:
        cq = d_q.get(label, 0.0)
        cc = d_c.get(label, 0.0)
        if abs(cq - cc) > tol:
            mismatches.append((label, cq, cc))

    print(f"#terms Qiskit: {len(d_q)}")
    print(f"#terms custom: {len(d_c)}")

    if not mismatches:
        print("✅ All coefficients match within tolerance.")
    else:
        print("❌ Mismatches (label, qiskit, custom):")
        for label, cq, cc in mismatches:
            print(f"  {label:>6s} : {cq:.12g} (qiskit)  vs  {cc:.12g} (custom)")

    return d_q, d_c


## VQE

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit.quantum_info import SparsePauliOp, Operator, Statevector
from qiskit.circuit.library import efficient_su2
from qiskit.primitives import StatevectorEstimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms.utils import algorithm_globals

# 1. Setup Consistency (Fix random seed for reproducible results)
algorithm_globals.random_seed = 1234
np.random.seed(1234)

# 2. Setup Physics
L = 1.0
N = 16  # 4 qubits
grid = GridDiscretization(L=L, N=N)

# Build Hamiltonian
H_matrix_physical = grid.build_hamiltonian()
H_matrix_padded   = grid.pad_to_qubit_dimension(H_matrix_physical)
qubit_op = SparsePauliOp.from_operator(Operator(H_matrix_padded))

print(f"System: Infinite Square Well (L={L}, N={N} grid points)")
print(f"Hamiltonian: {qubit_op.num_qubits} qubits, {len(qubit_op)} Pauli terms")

# 3. Setup VQE Components
# Ansatz: A parameterized circuit to explore the Hilbert space
ansatz = efficient_su2(num_qubits=grid.n_qubits, reps=3, entanglement='linear')
# Decompose the ansatz to see the individual gates (Rotations + CNOTs)
# 'mpl' style makes it a pretty image using Matplotlib
display(ansatz.decompose().draw(output='mpl'))
# Optimizer: COBYLA is good for noiseless statevector simulations
optimizer = COBYLA(maxiter=1000)

# Estimator: Calculates <psi|H|psi> using matrix multiplication (exact)
estimator = StatevectorEstimator()

# 4. Run VQE
# --- FIX START: Force identical starting parameters ---
initial_point = np.random.random(ansatz.num_parameters) # <--- ADDED THIS (Explicit start point)
vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point) # <--- CHANGED THIS (Added initial_point arg)
# --- FIX END ---

result = vqe.compute_minimum_eigenvalue(operator=qubit_op)

# 5. Output Numerical Results
print("\n--- VQE Results ---")
print(f"VQE Ground State Energy: {result.eigenvalue.real:.5f}")
E_exact, psi_exact = grid.analytical_infinite_well(n=1)
print(f"Exact Ground State Energy: {E_exact:.5f}")
print(f"Error: {abs(result.eigenvalue.real - E_exact):.5e}")

# 6. VISUALIZATION (Comparison of Statevectors)
# Reconstruct the VQE state vector from the optimal parameters
final_circuit = ansatz.assign_parameters(result.optimal_parameters)
vqe_vector = Statevector(final_circuit).data

# Fix Global Phase: If VQE found the negative version of the wave, flip it back
if np.sign(vqe_vector[0].real) != np.sign(psi_exact[0]):
    vqe_vector *= -1

# Calculate overlap (fidelity)
overlap = np.abs(np.dot(psi_exact.conj(), vqe_vector))**2
print(f"Wavefunction Fidelity: {overlap:.5f}")

# Plot
plt.figure(figsize=(10, 6))
plt.plot(grid.x_grid, psi_exact, 'b-', label='Analytical Exact Solution', linewidth=2)
plt.plot(grid.x_grid, vqe_vector.real, 'ro', label='VQE Result', markersize=8)
plt.title(f"Ground State Wavefunction Comparison (N={grid.N})")
plt.xlabel("Position (x)")
plt.ylabel("Wavefunction Amplitude ψ(x)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### VQE comparison

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit.quantum_info import SparsePauliOp, Operator, Statevector
from qiskit.circuit.library import efficient_su2
from qiskit.primitives import StatevectorEstimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms.utils import algorithm_globals



# Setup Physics
L = 1.0
N = 16  # 4 qubits
# ASSUMPTION: GridDiscretization class is defined in a previous cell
grid = GridDiscretization(L=L, N=N)

# Build Hamiltonian
H_matrix_physical = grid.build_hamiltonian()
H_matrix_padded   = grid.pad_to_qubit_dimension(H_matrix_physical)
qubit_op = SparsePauliOp.from_operator(Operator(H_matrix_padded))

# --- COMPARISON TARGETS ---
# 1. Analytical Solution (True Physics)
E_analytical, psi_exact = grid.analytical_infinite_well(n=1)
# 2. Grid Exact Solution (Matrix Limit)
E_grid_exact = np.min(np.linalg.eigvalsh(H_matrix_padded)).real

print(f"System: Infinite Square Well (L={L}, N={N} grid points)")
print(f"Analytical Exact Energy (Target): {E_analytical:.5f}")
print(f"Grid Exact Energy (Target): {E_grid_exact:.5f}")


reps_list = [1, 2, 3]
entanglement_list = ['linear', 'full', 'circular', 'sca']
NUM_TRIALS = 5  # 5 runs per configuration

# Optimizer Setup
optimizer = COBYLA(maxiter=1000, tol=0.001)
estimator = StatevectorEstimator()

results_stats = []
best_config_name = None
min_mean_error = float('inf')
best_single_run_result = None # To store the specific vector of the absolute best run

print(f"\n--- Starting VQE Benchmark ({NUM_TRIALS} trials per config) ---")

base_seed = 1000
config_idx = 0

for ent in entanglement_list:
    for reps in reps_list:
        config_idx += 1
        label = f"{ent}, reps={reps}"
        print(f"Testing: {label} ... ", end="")

        trial_errors = []
        trial_fidelities = []
        trial_evals = []

        best_run_in_group = None
        min_error_in_group = float('inf')

        # --- THE INNER TRIAL LOOP ---
        for i in range(NUM_TRIALS):
            # Unique seed for every trial
            # This ensures statistical independence
            seed = base_seed + 1000*config_idx + i

            #  Set seeds for BOTH Qiskit and Numpy ---
            algorithm_globals.random_seed = seed
            np.random.seed(seed)

            # 1. Configure Ansatz (New random parameters)
            ansatz = efficient_su2(num_qubits=grid.n_qubits, reps=reps, entanglement=ent)

            # Explicitly generate the start point ---
            initial_point = np.random.random(ansatz.num_parameters)

            # 2. Run VQE (Pass the explicit point)
            vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point)

            result = vqe.compute_minimum_eigenvalue(operator=qubit_op)

            # 3. Calculate Metrics
            # Compare against GRID exact for optimizer benchmarking
            opt_error = abs(result.eigenvalue.real - E_grid_exact)

            # Reconstruct statevector
            final_circuit = ansatz.assign_parameters(result.optimal_parameters)
            vqe_vector = Statevector(final_circuit).data

            # Fix Global Phase
            if np.sign(vqe_vector[0].real) != np.sign(psi_exact[0]):
                vqe_vector *= -1

            fidelity = np.abs(np.dot(psi_exact.conj(), vqe_vector))**2

            # Collect Stats
            trial_errors.append(opt_error)
            trial_fidelities.append(fidelity)
            trial_evals.append(result.cost_function_evals)

            # Keep track of the best single run within this group (for plotting later)
            if opt_error < min_error_in_group:
                min_error_in_group = opt_error
                best_run_in_group = {
                    'label': label,
                    'vector': vqe_vector,
                    'ansatz': ansatz,
                    'error': opt_error
                }

        # --- AGGREGATE STATS ---
        mean_error = np.mean(trial_errors)
        std_error = np.std(trial_errors)
        mean_fid = np.mean(trial_fidelities)
        mean_evals = np.mean(trial_evals)

        print(f"Mean Error: {mean_error:.2e} (±{std_error:.1e})")

        results_stats.append({
            'label': label,
            'mean_error': mean_error,
            'std_error': std_error,
            'mean_fidelity': mean_fid,
            'std_fidelity': np.std(trial_fidelities),
            'mean_evals': mean_evals,
            'best_run': best_run_in_group
        })

        # Determine the "Winning Configuration" based on MEAN error
        if mean_error < min_mean_error:
            min_mean_error = mean_error
            best_config_name = label
            # We save the best single run from the winning group to plot the wavefunction
            best_single_run_result = best_run_in_group

print(f"\nWinner (Best Consistency): {best_config_name} with Mean Error {min_mean_error:.2e}")

# --- VISUALIZATION 1: Statistical Comparison ---
labels = [d['label'] for d in results_stats]
mean_errors = [d['mean_error'] for d in results_stats]
std_errors = [d['std_error'] for d in results_stats]
mean_fids = [d['mean_fidelity'] for d in results_stats]
std_fids = [d['std_fidelity'] for d in results_stats]
mean_evals = [d['mean_evals'] for d in results_stats]

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

# Plot Errors with Error Bars
# yerr=std_errors adds the black "reliability" bars
ax1.bar(labels, mean_errors, yerr=std_errors, capsize=5, color='salmon', alpha=0.7)
ax1.set_title(f"Mean Optimization Error (over {NUM_TRIALS} seeds)")
ax1.set_ylabel("|E_vqe - E_grid_exact|")
ax1.set_yscale('log')
ax1.tick_params(axis='x', rotation=90)
ax1.grid(True, axis='y', alpha=0.3)

# Plot Fidelities with Error Bars
ax2.errorbar(labels, mean_fids, yerr=std_fids, fmt='bo-', linewidth=2, capsize=5)
ax2.set_title(f"Mean Fidelity (over {NUM_TRIALS} seeds)")
ax2.set_ylabel("Overlap |<ψ|ψ_vqe>|²")
ax2.set_ylim(0, 1.05)
ax2.tick_params(axis='x', rotation=90)
ax2.grid(True, alpha=0.3)

# Plot Efficiency (Evaluations)
ax3.bar(labels, mean_evals, color='green', alpha=0.7)
ax3.set_title("Mean Cost (Evaluations)")
ax3.set_ylabel("Avg Function Calls")
ax3.axhline(y=1000, color='r', linestyle='--', label='Max Iter Limit')
ax3.tick_params(axis='x', rotation=90)
ax3.grid(True, axis='y', alpha=0.3)
ax3.legend()

plt.tight_layout()
plt.show()

# --- VISUALIZATION 2: Best Wavefunction ---
# We plot the single best run from the statistically best configuration
print(f"\n--- Plotting Best Result from Winner ({best_config_name}) ---")
display(best_single_run_result['ansatz'].decompose().draw(output='mpl'))

plt.figure(figsize=(10, 6))
plt.plot(grid.x_grid, psi_exact, 'b-', label='Analytical Exact Solution', linewidth=2, alpha=0.6)
plt.plot(grid.x_grid, best_single_run_result['vector'].real, 'ro', label=f"Best Run of {best_config_name}", markersize=8)
plt.title(f"Ground State: Best Run of Winning Config (N={grid.N})")
plt.xlabel("Position (x)")
plt.ylabel("Wavefunction Amplitude ψ(x)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## VQD

In [ ]:
import numpy as np
import scipy.linalg as la
import pandas as pd
import matplotlib.pyplot as plt
from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit.primitives import StatevectorEstimator
from qiskit_algorithms.optimizers import COBYLA
from qiskit.circuit.library import efficient_su2  # no deprecation spam
from qiskit_algorithms import VQD
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit_algorithms.utils import algorithm_globals
from qiskit.primitives import StatevectorSampler
from qiskit.quantum_info import Statevector


# Setup Physics
L = 1.0
N = 16  # 4 qubits
# ASSUMPTION: GridDiscretization class is defined in a previous cell
grid = GridDiscretization(L=L, N=N)
optimizer = COBYLA(maxiter=1000, tol=0.001)
estimator = StatevectorEstimator()

# Build Hamiltonian
H_matrix_physical = grid.build_hamiltonian()
H_matrix_padded   = grid.pad_to_qubit_dimension(H_matrix_physical)
qubit_op = SparsePauliOp.from_operator(Operator(H_matrix_padded))

# ==========================================
# 1. SETUP & EXACT REFERENCE
# ==========================================
k = 3
num_trials = 5
base_seed = 42

print("Calculating exact eigenvalues from H_matrix_padded...")
exact_eigs = np.sort(la.eigvalsh(H_matrix_padded).real)
exact_grid_energies = exact_eigs[:k]
print(f"Target (Grid Exact) Energies (lowest {k}): {exact_grid_energies}")

# ==========================================
# 2. DEFINE SEARCH SPACE
# ==========================================
entanglement_list = ["linear", "full", "circular", "sca"]
reps_list = [1, 2, 3, 4]

fidelity = ComputeUncompute(sampler=StatevectorSampler())
vqd_summary_data = []

print(f"\n--- Starting VQD Grid Search ({num_trials} trials each) ---")
print(f"{'Configuration':<25} | {'Mean Error':<12} | {'Std Dev':<12} | {'Failures':<8} | {'Best Single Run':<14}")
print("-" * 90)

config_idx = 0

for ent in entanglement_list:
    for r in reps_list:
        config_name = f"ent={ent}, reps={r}"
        config_idx += 1

        trial_errors = []
        failures = 0

        best_run_in_this_config = None
        min_error_in_this_config = float("inf")

        for t in range(num_trials):
            # make seeds unique across configs too
            current_seed = base_seed + 1000 * config_idx + t
            algorithm_globals.random_seed = current_seed
            np.random.seed(current_seed)

            try:
                ansatz = efficient_su2(
                    num_qubits=grid.n_qubits,
                    reps=r,
                    entanglement=ent
                )

                # ✅ IMPORTANT FIX: one initial point per eigenstate
                # explore angles more broadly than [0,1)
                initial_points = [
                    2 * np.pi * np.random.random(ansatz.num_parameters)
                    for _ in range(k)
                ]

                # betas penalize overlap with earlier states (k-1 terms)
                # (increasing schedule often works better than constant)
                betas = [10.0 * (j + 1) for j in range(k - 1)]

                vqd = VQD(
                    estimator=estimator,
                    fidelity=fidelity,
                    ansatz=ansatz,
                    optimizer=optimizer,
                    k=k,
                    betas=betas,
                    initial_point=initial_points
                )

                result = vqd.compute_eigenvalues(operator=qubit_op)

                # energies + params come in "deflation step order"; reorder by energy for scoring/plotting
                energies = np.array([ev.real for ev in result.eigenvalues], dtype=float)
                order = np.argsort(energies)

                energies_sorted = energies[order][:k]
                params_sorted = [result.optimal_parameters[i] for i in order[:k]]

                # error metric (sum of abs energy diffs)
                total_error = float(np.sum(np.abs(energies_sorted - exact_grid_energies)))
                trial_errors.append(total_error)

                if total_error < min_error_in_this_config:
                    min_error_in_this_config = total_error
                    best_run_in_this_config = {
                        "params": params_sorted,          # list of k dicts
                        "energies": energies_sorted,      # sorted by energy
                        "ansatz": ansatz,
                        "seed": current_seed,
                        "betas": betas
                    }

            except Exception:
                failures += 1
                continue

        if trial_errors:
            mean_error = float(np.mean(trial_errors))
            std_error = float(np.std(trial_errors))

            print(f"{config_name:<25} | {mean_error:.2e}   | {std_error:.2e}   | {failures:<8d} | {min_error_in_this_config:.2e}")

            vqd_summary_data.append({
                "Config": config_name,
                "Mean_Error": mean_error,
                "Std_Error": std_error,
                "Failures": failures,
                "Best_Run_Error": min_error_in_this_config,
                "Best_Run": best_run_in_this_config
            })

# ==========================================
# 3. ANALYSIS & SELECTION
# ==========================================
if not vqd_summary_data:
    print("\nNo successful VQD runs.")
else:
    df_vqd = pd.DataFrame(vqd_summary_data).sort_values(by="Mean_Error").reset_index(drop=True)
    best_config = df_vqd.iloc[0]
    best_run = best_config["Best_Run"]

    print("\n=== BEST VQD CONFIGURATION (Based on Mean Error) ===")
    print(f"Config:      {best_config['Config']}")
    print(f"Mean Error:  {best_config['Mean_Error']:.5e}")
    print(f"Std Dev:     {best_config['Std_Error']:.5e}")
    print(f"Failures:    {int(best_config['Failures'])}")
    print(f"Best Seed:   {best_run['seed']}")
    print(f"Betas:       {best_run['betas']}")
    print(f"Best Energies (sorted): {best_run['energies']}")
    print(f"Exact Energies:         {exact_grid_energies}")

    # ==========================================
    # 4. VISUALIZATION OF BEST RUN
    # ==========================================
    best_ansatz = best_run["ansatz"]
    best_params_list = best_run["params"]

    plt.figure(figsize=(10, 6))
    x = grid.x_grid

    for i in range(k):
        # build statevector
        sv_full = Statevector(best_ansatz.assign_parameters(best_params_list[i])).data

        # ✅ robust: only take the physical subspace (first N components)
        N_phys = grid.N
        sv_phys = sv_full[:N_phys].copy()

        # leakage into padded states (useful debug metric)
        leakage = float(1.0 - np.sum(np.abs(sv_phys) ** 2))

        # compare to analytical wavefunction for same n
        _, psi_exact = grid.analytical_infinite_well(n=i + 1)

        # ✅ global phase alignment (works even if complex)
        phase = np.vdot(psi_exact, sv_phys)
        if np.abs(phase) > 1e-12:
            sv_phys *= np.exp(-1j * np.angle(phase))

        # optional: renormalize physical part for plotting (makes shape comparison clearer)
        norm = np.linalg.norm(sv_phys)
        if norm > 1e-12:
            sv_phys /= norm

        plt.plot(x, sv_phys.real, marker='o', markersize=4, linestyle='None',
                 label=f'VQD n={i+1} (leak={leakage:.1e})')
        plt.plot(x, psi_exact, linestyle='-', alpha=0.3)

    plt.title(
        f"Best VQD Result: {best_config['Config']} "
        f"(Mean Err: {best_config['Mean_Error']:.2e}, Best Err: {best_config['Best_Run_Error']:.2e})"
    )
    plt.xlabel("Position x")
    plt.ylabel("Wavefunction amplitude")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

    # show the summary table (optional)
    display(df_vqd[["Config", "Mean_Error", "Std_Error", "Failures", "Best_Run_Error"]])


Calculating exact eigenvalues from H_matrix_padded...
Target (Grid Exact) Energies (lowest 3): [ 4.92077419 19.5155257  43.28724777]

--- Starting VQD Grid Search (5 trials each) ---
Configuration             | Mean Error   | Std Dev      | Failures | Best Single Run
------------------------------------------------------------------------------------------
ent=linear, reps=1        | 2.82e+01   | 9.44e+00   | 0        | 2.03e+01


## SSVQE

### SSVQE using Matrix Multiplication

In [ ]:
# ============================================================
# SSVQE CONTROL-GROUP (Standalone Cell) Matrix multiplication mode
# - Builds grid + Hamiltonian + qubit_op inside this cell
# - Grid-search over entanglement/reps
# - num_trials seeds per config
# - Targets k = 3 states (ground + 1st + 2nd excited)
# ============================================================

import numpy as np
import scipy.linalg as la
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize

from qiskit.circuit.library import efficient_su2
from qiskit.quantum_info import Operator, SparsePauliOp, Statevector
from qiskit_algorithms.utils import algorithm_globals

# ----------------------------
# Minimal GridDiscretization (self-contained)
# ----------------------------
class GridDiscretization:
    def __init__(self, L: float, N: int, hbar: float = 1.0, m: float = 1.0):
        self.L = L
        self.N = N
        self.hbar = hbar
        self.m = m
        self.dx = L / (N + 1)
        self.x_grid = np.linspace(self.dx, L - self.dx, N)
        self.n_qubits = int(np.ceil(np.log2(N)))
        self.hilbert_space_dim = 2 ** self.n_qubits

    def build_kinetic_energy(self) -> np.ndarray:
        coeff = -self.hbar**2 / (2 * self.m * self.dx**2)
        T = np.zeros((self.N, self.N), dtype=float)
        for i in range(self.N):
            T[i, i] = -2.0
            if i > 0:       T[i, i-1] = 1.0
            if i < self.N-1: T[i, i+1] = 1.0
        return coeff * T

    def build_hamiltonian(self) -> np.ndarray:
        # Infinite square well: V=0 inside, so H = T
        return self.build_kinetic_energy()

    def analytical_infinite_well(self, n: int):
        E_n = (n * np.pi * self.hbar)**2 / (2 * self.m * self.L**2)
        psi = np.sqrt(2 / self.L) * np.sin(n * np.pi * self.x_grid / self.L)
        psi_discrete = np.sqrt(self.dx) * psi  # discrete normalization convention
        return E_n, psi_discrete

    def pad_to_qubit_dimension(self, matrix: np.ndarray, penalty_factor: float = 1e3) -> np.ndarray:
        if matrix.shape[0] == self.hilbert_space_dim:
            return matrix

        padded = np.zeros((self.hilbert_space_dim, self.hilbert_space_dim), dtype=complex)
        n = matrix.shape[0]
        padded[:n, :n] = matrix.astype(complex)

        max_entry = np.max(np.abs(matrix))
        if max_entry == 0:
            max_entry = 1.0
        Lambda = penalty_factor * max_entry

        padded[n:, n:] = np.eye(self.hilbert_space_dim - n) * Lambda
        return padded


# ============================================================
# 1) SETUP (edit L, N if you want)
# ============================================================
L = 1.0
N = 16  # 4 qubits when N=16

k = 3
num_trials = 5
base_seed = 42

entanglement_list = ["linear", "full", "circular", "sca"]
reps_list = [1, 2, 3, 4]

maxiter = 1000

# SSVQE weights: strictly decreasing, length=k
# (tune these if excited states collapse to weird high-energy orthogonal states)
weights = [50.0, 5.0, 1.0]

print("Building grid + Hamiltonian ...")
grid = GridDiscretization(L=L, N=N)
H_matrix_physical = grid.build_hamiltonian()
H_matrix_padded = grid.pad_to_qubit_dimension(H_matrix_physical)
qubit_op = SparsePauliOp.from_operator(Operator(H_matrix_padded))

print(f"System: Infinite Square Well (L={L}, N={N})")
print(f"Qubits: {grid.n_qubits}, Hilbert dim: {2**grid.n_qubits}")
print(f"Hamiltonian Pauli terms: {len(qubit_op)}")

# ============================================================
# 2) EXACT REFERENCE (grid exact)
# ============================================================
exact_eigs = np.sort(la.eigvalsh(H_matrix_padded).real)
exact_grid_energies = exact_eigs[:k]
print(f"Target (Grid Exact) Energies (lowest {k}): {exact_grid_energies}")

# Precompute dense matrix ONCE for fast <psi|H|psi>
H_mat = qubit_op.to_matrix()

def energy(psi: np.ndarray) -> float:
    return float(np.vdot(psi, H_mat @ psi).real)

# k orthogonal input basis states |0>,|1>,|2> ... as n-bit strings
labels = [format(i, f"0{grid.n_qubits}b") for i in range(k)]
inputs = [Statevector.from_label(lbl) for lbl in labels]

print(f"SSVQE inputs: {labels}")
print(f"SSVQE weights: {weights}")
print(f"COBYLA maxiter: {maxiter}")

# ============================================================
# 3) GRID SEARCH
# ============================================================
ssvqe_summary_data = []

print(f"\n--- Starting SSVQE Grid Search ({num_trials} trials each) ---")
print(f"{'Configuration':<25} | {'Mean Error':<12} | {'Std Dev':<12} | {'Failures':<8} | {'Best Single Run':<14}")
print("-" * 95)

config_idx = 0

for ent in entanglement_list:
    for reps in reps_list:
        config_idx += 1
        config_name = f"ent={ent}, reps={reps}"

        trial_errors = []
        failures = 0

        best_run = None
        best_err = float("inf")

        for t in range(num_trials):
            seed = base_seed + 1000 * config_idx + t
            algorithm_globals.random_seed = seed
            np.random.seed(seed)

            try:
                ansatz = efficient_su2(num_qubits=grid.n_qubits, reps=reps, entanglement=ent)

                def ssvqe_cost(params):
                    bound = ansatz.assign_parameters(params)
                    psis = [inp.evolve(bound).data for inp in inputs]
                    Es = [energy(psi) for psi in psis]
                    return float(np.dot(weights, Es))

                init = 2 * np.pi * np.random.random(ansatz.num_parameters)
                res = minimize(ssvqe_cost, init, method="COBYLA", options={"maxiter": maxiter})

                bound_final = ansatz.assign_parameters(res.x)
                psis_final = [inp.evolve(bound_final).data for inp in inputs]
                Es_raw = np.array([energy(psi) for psi in psis_final], dtype=float)

                order = np.argsort(Es_raw)
                Es_sorted = Es_raw[order][:k]

                total_error = float(np.sum(np.abs(Es_sorted - exact_grid_energies)))
                trial_errors.append(total_error)

                if total_error < best_err:
                    best_err = total_error
                    best_run = {
                        "seed": seed,
                        "ansatz": ansatz,
                        "params": res.x,
                        "energies_raw": Es_raw,
                        "energies_sorted": Es_sorted,
                        "order": order,
                        "labels": labels,
                    }

            except Exception:
                failures += 1
                continue

        if trial_errors:
            mean_error = float(np.mean(trial_errors))
            std_error = float(np.std(trial_errors))

            print(f"{config_name:<25} | {mean_error:>10.3e} | {std_error:>10.3e} | {failures:<8d} | {best_err:>12.3e}")

            ssvqe_summary_data.append({
                "Config": config_name,
                "Mean_Error": mean_error,
                "Std_Error": std_error,
                "Failures": failures,
                "Best_Run_Error": best_err,
                "Best_Run": best_run
            })

# ============================================================
# 4) PICK BEST CONFIG + PLOT BEST RUN
# ============================================================
if not ssvqe_summary_data:
    print("\nNo successful SSVQE runs.")
else:
    df_ssvqe = pd.DataFrame(ssvqe_summary_data).sort_values(by="Mean_Error").reset_index(drop=True)
    display(df_ssvqe[["Config", "Mean_Error", "Std_Error", "Failures", "Best_Run_Error"]])

    best_config = df_ssvqe.iloc[0]
    best = best_config["Best_Run"]

    print("\n=== BEST SSVQE CONFIGURATION (Based on Mean Error) ===")
    print(f"Config:      {best_config['Config']}")
    print(f"Mean Error:  {best_config['Mean_Error']:.6e}")
    print(f"Std Dev:     {best_config['Std_Error']:.6e}")
    print(f"Failures:    {int(best_config['Failures'])}")
    print(f"Best Seed:   {best['seed']}")
    print(f"Energies raw (per input): {best['energies_raw']}")
    print(f"Energies sorted:          {best['energies_sorted']}")
    print(f"Exact energies:           {exact_grid_energies}")

    # Compute and print overlaps between the k outputs (should be ~0 due to unitarity)
    ansatz = best["ansatz"]
    params = best["params"]
    order = best["order"]

    bound_final = ansatz.assign_parameters(params)
    psis_final = [inp.evolve(bound_final).data for inp in inputs]
    selected = [psis_final[idx] for idx in order[:k]]

    print("\nOverlaps |<psi_i|psi_j>|^2 (should be ~0):")
    for i in range(k):
        for j in range(i):
            ov = abs(np.vdot(selected[j], selected[i]))**2
            print(f"  ({j+1},{i+1}) = {ov:.3e}")

    # Plot wavefunctions in physical subspace
    plt.figure(figsize=(10, 6))
    x = grid.x_grid

    for i in range(k):
        psi_full = selected[i]
        psi_phys = psi_full[:grid.N].copy()

        leakage = float(1.0 - np.sum(np.abs(psi_phys)**2))

        _, psi_exact = grid.analytical_infinite_well(n=i+1)
        ph = np.vdot(psi_exact, psi_phys)
        if abs(ph) > 1e-12:
            psi_phys *= np.exp(-1j * np.angle(ph))

        nrm = np.linalg.norm(psi_phys)
        if nrm > 1e-12:
            psi_phys /= nrm

        plt.plot(x, psi_phys.real, marker="o", markersize=4, linestyle="None",
                 label=f"SSVQE n={i+1} (leak={leakage:.1e})")
        plt.plot(x, psi_exact, linestyle="-", alpha=0.25)

    plt.title(f"Best SSVQE: {best_config['Config']} (Mean Err: {best_config['Mean_Error']:.2e})")
    plt.xlabel("Position x")
    plt.ylabel("Wavefunction amplitude")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()


### SSVQE using statevectorEstimator Version

In [ ]:
# ============================================================
# SSVQE CONTROL-GROUP (StatevectorEstimator version)
# - Standalone: defines GridDiscretization, builds H, builds qubit_op
# - Uses StatevectorEstimator to compute <H> (no qubit_op.to_matrix())
# - Grid-search over entanglement/reps + num_trials seeds
# - Targets k=3 (ground + 1st + 2nd excited)
# ============================================================

import numpy as np
import scipy.linalg as la
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize

from qiskit import QuantumCircuit
from qiskit.circuit.library import efficient_su2
from qiskit.quantum_info import Operator, SparsePauliOp, Statevector
from qiskit.primitives import StatevectorEstimator
from qiskit_algorithms.utils import algorithm_globals


# ----------------------------
# Minimal GridDiscretization
# ----------------------------
class GridDiscretization:
    def __init__(self, L: float, N: int, hbar: float = 1.0, m: float = 1.0):
        self.L = float(L)
        self.N = int(N)
        self.hbar = float(hbar)
        self.m = float(m)

        self.dx = self.L / (self.N + 1)
        self.x_grid = np.linspace(self.dx, self.L - self.dx, self.N)

        self.n_qubits = int(np.ceil(np.log2(self.N)))
        self.hilbert_space_dim = 2 ** self.n_qubits

    def build_hamiltonian(self) -> np.ndarray:
        # 1D infinite square well: H = T (second-derivative finite difference), V=0
        coeff = -self.hbar**2 / (2 * self.m * self.dx**2)
        T = np.zeros((self.N, self.N), dtype=float)
        for i in range(self.N):
            T[i, i] = -2.0
            if i > 0:
                T[i, i - 1] = 1.0
            if i < self.N - 1:
                T[i, i + 1] = 1.0
        return coeff * T

    def pad_to_qubit_dimension(self, matrix: np.ndarray, penalty_factor: float = 1e3) -> np.ndarray:
        if matrix.shape[0] == self.hilbert_space_dim:
            return matrix.astype(complex)

        padded = np.zeros((self.hilbert_space_dim, self.hilbert_space_dim), dtype=complex)
        n = matrix.shape[0]
        padded[:n, :n] = matrix.astype(complex)

        max_entry = np.max(np.abs(matrix)) if np.max(np.abs(matrix)) != 0 else 1.0
        Lambda = penalty_factor * max_entry
        padded[n:, n:] = np.eye(self.hilbert_space_dim - n) * Lambda
        return padded

    def analytical_infinite_well(self, n: int):
        # Continuous eigenfunction sampled on the grid; return a discretized vector for plotting/overlap
        E_n = (n * np.pi * self.hbar) ** 2 / (2 * self.m * self.L**2)
        psi = np.sqrt(2 / self.L) * np.sin(n * np.pi * self.x_grid / self.L)
        psi_discrete = np.sqrt(self.dx) * psi
        return float(E_n), psi_discrete


def prep_basis_circuit(label: str, n_qubits: int) -> QuantumCircuit:
    """
    Prepare |label> from |0...0>.
    Qiskit convention: rightmost bit corresponds to qubit 0.
    """
    qc = QuantumCircuit(n_qubits)
    for q, bit in enumerate(reversed(label)):
        if bit == "1":
            qc.x(q)
    return qc


# ============================================================
# 1) SETTINGS
# ============================================================
L = 1.0
N = 16

k = 3
num_trials = 5
base_seed = 42

entanglement_list = ["linear", "full", "circular", "sca"]
reps_list = [1, 2, 3, 4]

maxiter = 800

# strictly decreasing weights, length=k
weights = [50.0, 5.0, 1.0]
assert len(weights) == k and all(weights[i] > weights[i + 1] for i in range(k - 1))

# k orthogonal computational basis inputs: |0>,|1>,|2>...
labels = [format(i, f"0{int(np.ceil(np.log2(N)))}b") for i in range(k)]


# ============================================================
# 2) BUILD GRID + HAMILTONIAN + QUBIT OPERATOR
# ============================================================
print("Building grid + Hamiltonian...")
grid = GridDiscretization(L=L, N=N)

H_matrix_physical = grid.build_hamiltonian()
H_matrix_padded = grid.pad_to_qubit_dimension(H_matrix_physical)

# This is your line: dense matrix -> Qiskit Operator -> Pauli-sum (SparsePauliOp)
qubit_op = SparsePauliOp.from_operator(Operator(H_matrix_padded))

print(f"System: Infinite Square Well (L={L}, N={N})")
print(f"Qubits: {grid.n_qubits}, Hilbert dim: {2**grid.n_qubits}")
print(f"Pauli terms: {len(qubit_op)}")

# Exact reference energies from padded matrix (small enough to diagonalize here)
exact_eigs = np.sort(la.eigvalsh(H_matrix_padded).real)
exact_grid_energies = exact_eigs[:k]
print(f"Target (Grid Exact) Energies (lowest {k}): {exact_grid_energies}")

print(f"SSVQE inputs: {labels}")
print(f"Weights: {weights}, COBYLA maxiter: {maxiter}")

estimator = StatevectorEstimator()


def energies_via_statevectorestimator(circuits, params):
    """
    Returns array of expectation values <H> for each circuit at 'params'
    using StatevectorEstimator (no dense H matrix).
    """
    params = np.asarray(params, dtype=float)

    # Try passing parameter_values as 1D; if your Qiskit expects 2D, fallback.
    try:
        pubs = [(qc, qubit_op, params) for qc in circuits]
        job = estimator.run(pubs)
        res = job.result()
        return np.array([float(res[i].data.evs) for i in range(len(circuits))], dtype=float)
    except Exception:
        params2 = params.reshape(1, -1)
        pubs = [(qc, qubit_op, params2) for qc in circuits]
        job = estimator.run(pubs)
        res = job.result()
        return np.array([float(res[i].data.evs) for i in range(len(circuits))], dtype=float)


# ============================================================
# 3) GRID SEARCH
# ============================================================
summary = []

print(f"\n--- Starting SSVQE Grid Search ({num_trials} trials each) ---")
print(f"{'Configuration':<25} | {'Mean Error':<12} | {'Std Dev':<12} | {'Failures':<8} | {'Best Single Run':<14}")
print("-" * 95)

config_idx = 0

for ent in entanglement_list:
    for reps in reps_list:
        config_idx += 1
        config_name = f"ent={ent}, reps={reps}"

        trial_errors = []
        failures = 0

        best_run = None
        best_err = float("inf")

        # Build ansatz + the k input circuits ONCE per config
        ansatz = efficient_su2(num_qubits=grid.n_qubits, reps=reps, entanglement=ent)

        # circuits[i] prepares |labels[i]> then applies ansatz
        circuits = []
        for lbl in labels:
            prep = prep_basis_circuit(lbl, grid.n_qubits)
            circuits.append(prep.compose(ansatz))

        for t in range(num_trials):
            seed = base_seed + 1000 * config_idx + t
            algorithm_globals.random_seed = seed
            np.random.seed(seed)

            try:
                def cost(params):
                    Es = energies_via_statevectorestimator(circuits, params)
                    return float(np.dot(weights, Es))

                init = 2 * np.pi * np.random.random(ansatz.num_parameters)
                res = minimize(cost, init, method="COBYLA", options={"maxiter": maxiter})

                Es_raw = energies_via_statevectorestimator(circuits, res.x)

                order = np.argsort(Es_raw)
                Es_sorted = Es_raw[order][:k]

                total_error = float(np.sum(np.abs(Es_sorted - exact_grid_energies)))
                trial_errors.append(total_error)

                if total_error < best_err:
                    best_err = total_error
                    best_run = {
                        "seed": seed,
                        "ansatz": ansatz,
                        "params": np.asarray(res.x, dtype=float),
                        "energies_raw": Es_raw,
                        "order": order,
                        "labels": labels,
                        "ent": ent,
                        "reps": reps,
                    }

            except Exception:
                failures += 1
                continue

        if trial_errors:
            mean_error = float(np.mean(trial_errors))
            std_error = float(np.std(trial_errors))

            print(f"{config_name:<25} | {mean_error:>10.3e} | {std_error:>10.3e} | {failures:<8d} | {best_err:>12.3e}")

            summary.append({
                "Config": config_name,
                "Mean_Error": mean_error,
                "Std_Error": std_error,
                "Failures": failures,
                "Best_Run_Error": best_err,
                "Best_Run": best_run
            })

# ============================================================
# 4) PICK BEST CONFIG + PLOT BEST RUN
# ============================================================
if not summary:
    print("\nNo successful SSVQE runs.")
else:
    df = pd.DataFrame(summary).sort_values(by="Mean_Error").reset_index(drop=True)
    display(df[["Config", "Mean_Error", "Std_Error", "Failures", "Best_Run_Error"]])

    best_config = df.iloc[0]
    best = best_config["Best_Run"]

    print("\n=== BEST SSVQE CONFIGURATION (Based on Mean Error) ===")
    print(f"Config:      {best_config['Config']}")
    print(f"Mean Error:  {best_config['Mean_Error']:.6e}")
    print(f"Std Dev:     {best_config['Std_Error']:.6e}")
    print(f"Failures:    {int(best_config['Failures'])}")
    print(f"Best Seed:   {best['seed']}")
    print(f"Energies raw (per input): {best['energies_raw']}")
    print(f"Exact energies:           {exact_grid_energies}")

    # Build statevectors for plotting (use Statevector evolution for convenience)
    ansatz = best["ansatz"]
    params = best["params"]
    order = best["order"]

    # bind parameters robustly
    param_map = dict(zip(list(ansatz.parameters), params))
    bound_ansatz = ansatz.assign_parameters(param_map)

    # output states correspond to ansatz acting on |label_i>
    svs = [Statevector.from_label(lbl).evolve(bound_ansatz).data for lbl in labels]
    selected = [svs[idx] for idx in order[:k]]  # lowest energies first

    plt.figure(figsize=(10, 6))
    x = grid.x_grid

    for i in range(k):
        sv_full = selected[i]
        sv_phys = sv_full[:grid.N].copy()
        leakage = float(1.0 - np.sum(np.abs(sv_phys) ** 2))

        _, psi_exact = grid.analytical_infinite_well(n=i + 1)

        # align global phase for nicer comparison
        ph = np.vdot(psi_exact, sv_phys)
        if abs(ph) > 1e-12:
            sv_phys *= np.exp(-1j * np.angle(ph))

        # renormalize physical part for plotting
        nrm = np.linalg.norm(sv_phys)
        if nrm > 1e-12:
            sv_phys /= nrm

        plt.plot(x, sv_phys.real, marker="o", markersize=4, linestyle="None",
                 label=f"SSVQE n={i+1} (leak={leakage:.1e})")
        plt.plot(x, psi_exact, linestyle="-", alpha=0.25)

    plt.title(f"Best SSVQE: {best_config['Config']} (Mean Err: {best_config['Mean_Error']:.2e})")
    plt.xlabel("Position x")
    plt.ylabel("Wavefunction amplitude")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
